In [ ]:
%autoreload 2

In [ ]:
import logging 
logging.basicConfig()
log = logging.getLogger("blocking").setLevel("DEBUG")
log = logging.getLogger("sql").setLevel("DEBUG")

In [ ]:
from pyspark.context import SparkContext, SparkConf
from pyspark.sql import SparkSession, Window
from pyspark.sql.types import StructType
import pyspark.sql.functions as f

# WARNING:
# These config options are appropriate only if you're running Spark locally!!!
conf=SparkConf()
conf.set('spark.driver.memory', '8g')
conf.set("spark.sql.shuffle.partitions", "8") 

sc = SparkContext.getOrCreate(conf=conf)

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [ ]:
import sys 
sys.path.append("sparklink")
from sql import *
from blocking import *

In [ ]:
df = spark.read.parquet("data/fake_1000.parquet")
df = df.dropDuplicates()
df.show()

In [ ]:
blocking_rules = []

blocking_rules.append('l.first_name = r.first_name  and  l.dob = r.dob')
blocking_rules.append('l.first_name = r.first_name  and  l.city = r.city')
blocking_rules.append('l.first_name = r.first_name  and  l.email = r.email')

blocking_rules.append('l.surname = r.surname  and  l.dob = r.dob')
blocking_rules.append('l.surname = r.surname  and  l.city = r.city')
blocking_rules.append('l.surname = r.surname  and  l.email = r.email')

blocking_rules.append('l.dob = r.dob  and  l.city = r.city')
blocking_rules.append('l.dob = r.dob  and  l.email = r.email')

blocking_rules.append('l.city = r.city  and  l.email = r.email')

In [ ]:
df_comparison = block_using_rules(df, blocking_rules, spark=spark)

In [ ]:
gamma_settings = {"first_name": 3, "surname": 2, "dob": 2, "city": 2, "email":2}

In [ ]:
df_comparison.show()